In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import seaborn as sb
from matplotlib import pyplot as plt
from matplotlib import cm, colors

In [ ]:
prefix = "DD25"       # set to TDD for Thawing Degree Days

config = {
    "TDD": {
        "deltaT": 32
    },
    "DD20": {
        "deltaT": 20
    },
    "DD25": {
        "deltaT": 25
    },
}

PROJPATH = Path().resolve().parent
breakupdata = PROJPATH / "data/breakupdata/derived"
stationdata = PROJPATH / f"data/weatherstations/ACIS/{prefix}/dd_climatologies"

In [ ]:
stations_ddclim = stationdata.glob("*.csv")
stations_ddclim_list = list(stations_ddclim)

## Prototyping

In [ ]:
testdata = pd.read_csv(stations_ddclim_list[3], skiprows=4, index_col=0)
testdata

In [ ]:
stationname = stations_ddclim_list[3].stem[:-14]
testdf = pd.DataFrame(testdata['dd'].cumsum())
testdf.columns = [stationname]
testdf

In [ ]:
sb.lineplot(data=testdf[stationname])

### Loop

In [ ]:
alldd = []
for fpth in stations_ddclim_list:
    stationname = fpth.stem[:-14]
    testdata = pd.read_csv(fpth, skiprows=4, index_col=0)
    testdf = pd.DataFrame(testdata['dd'].cumsum())
    testdf.columns = [stationname]
    alldd.append(testdf)

In [ ]:
clrs = plt.cm.rainbow(np.linspace(0, 1, len(alldd)))
clrs = plt.cm.tab20b.colors + plt.cm.tab20c.colors

fig, ax = plt.subplots(figsize=(15, 10))
pd.concat(alldd, axis=1).plot(ax=ax, legend=False, color=clrs)
ax.legend(fontsize=9, ncol=2, loc='upper left')
ax.set_xlabel('Days since March 1')
ax.set_ylabel(f'Degree days > {config[prefix]["deltaT"]} (F)')


In [ ]:
allcumulddds = pd.concat(alldd, axis=1)
allcumulddds

In [ ]:
outpath = PROJPATH / f"data/weatherstations/ACIS/{prefix}/"
fn = "all_cumul_clim1991_2020.csv"

with open(outpath / fn, 'w') as dst:
    dst.write(f"# All climatologies for cumulative degree days > {config[prefix]['deltaT']}, 1991-2020\n")
    dst.write(f"# Cumulative degree days starting March 1 from ACIS\n")
    dst.write("#\n")
    pd.concat(alldd, axis=1).to_csv(dst, float_format='%.2f')